source of data: [The Japanese Mortality Database](https://www.ipss.go.jp/p-toukei/JMD/index-en.asp) > "Life expectancy at birth" tables<br>
wiki > [List of Japanese prefectures by life expectancy](https://en.wikipedia.org/wiki/List_of_Japanese_prefectures_by_life_expectancy) <i>[(alternative)](https://en.wikipedia.org/wiki/User:Lady3mlnm/List_of_Japanese_prefectures_by_life_expectancy_(alternative))</i> / [Продолжительность жизни в префектурах Японии](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_префектурах_Японии)<br />
[MapChart](https://www.mapchart.net/japan.html)<br />
see also: [Nippon.com](https://www.nippon.com/en/search.html?s=life%20expectancy)

In [2]:
import pandas as pd
# import math
import re
from collections import namedtuple

import sys
sys.path.append("..")
import mal_moduls_private.mal_total as mal

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
pd.options.display.min_rows = 6
pd.options.display.max_rows = 50
YEARS_SEL = list(range(2010, 2024))
SORTED_BY = [2020, 2023]
SELECTED_YEAR = 2020
CREATE_LEGEND_CODE = True

In [5]:
CountryGroup = namedtuple('CountryGroup', ['group_label', 'color', 'countries'])

<br>

In [7]:
pattern_region_general = re.compile(r'(?<=^[\d.]{3}).*(?=(.ken|.to|.fu), Life expectancy)')  # what is in the most cases, like '47.Okinawa.ken'
pattern_region_1       = re.compile(r'(?<=^[\d.]{3}).*(?=, Life expectancy)')  # case of '01.Hokkaido' in the file Jp_01.csv
pattern_region_0       = re.compile(r'(?<=^).*(?=, Life expectancy)')  # case of 'Japan' in the file Jp_01.csv

def load_data_for_region(region_nmb):
    df = pd.read_csv(f"data-LE_at_birth/Jp_{region_nmb:02}.csv", sep='\t', skiprows=2, index_col='Year')

    with open(f"data-LE_at_birth/Jp_{region_nmb:02}.csv", 'r') as fh:
        title_line = fh.readline()
        if region_nmb >= 2:
            fragment_name_found = re.search(pattern_region_general, title_line)
        elif region_nmb == 1:
            fragment_name_found = re.search(pattern_region_1, title_line)
        elif region_nmb == 0:
            fragment_name_found = re.search(pattern_region_0, title_line)
        else:
            raise ValueError(f"Not expected region number: {region_nmb}")
        title = fragment_name_found.group() if fragment_name_found else 'region name is not found'

    df.name = title
        
    return df, title


df_t = df_m = df_f = pd.DataFrame()
ls_titles = []

for region_nmb in range(48):
    df_current, title_current = load_data_for_region(region_nmb)

    df_t = pd.concat([df_t, df_current.loc[YEARS_SEL, 'Total']], axis='columns')
    df_m = pd.concat([df_m, df_current.loc[YEARS_SEL, 'Male']], axis='columns')
    df_f = pd.concat([df_f, df_current.loc[YEARS_SEL, 'Female']], axis='columns')
    ls_titles.append(title_current)

df_t.columns = df_m.columns = df_f.columns = ls_titles

df_t = df_t.T
df_m = df_m.T
df_f = df_f.T

df_t.head(3)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Japan,82.95,82.71,83.21,83.43,83.68,83.92,84.10,84.22,84.32,84.46,84.71,84.58,84.10,84.14
Hokkaido,82.70,82.83,82.98,83.19,83.42,83.63,83.69,83.99,83.98,84.07,84.32,83.98,83.57,83.62
Aomori,81.33,81.55,81.50,81.91,82.05,82.41,82.65,82.74,82.81,82.60,83.09,82.96,82.69,82.22


In [8]:
# for region in df_t.index.to_list():
#     print(f"    '{region}'\t: {{'jp': ('', ''), 'en': ('', ''), 'ru': ('', '')}},")

<br>

In [10]:
# sort values
df_t = pd.concat([df_t.loc[['Japan']],
                  df_t.drop(index='Japan').sort_values(by=SORTED_BY, ascending=False)])

ls_indexes_sorted = df_t.index.to_list()
df_m = df_m.loc[ls_indexes_sorted]
df_f = df_f.loc[ls_indexes_sorted]

df_t.head(3)

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
Japan,82.95,82.71,83.21,83.43,83.68,83.92,84.10,84.22,84.32,84.46,84.71,84.58,84.10,84.14
Shiga,83.66,83.84,83.98,84.20,84.52,84.63,84.99,84.96,85.15,85.35,85.71,85.60,85.09,85.19
Kyoto,83.50,83.54,83.62,83.88,84.18,84.49,84.74,84.89,85.07,85.20,85.54,85.39,84.64,84.89


<br>

In [12]:
# just for interest, explore results: determine regions with max and min values, and also look at specific regions
mal.min_and_max_values(df_t, row_center=['Japan', 'Okinawa'], nmb=3, max_lng=9)

Number of records: 48


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
max,84.04 -Nagano,83.86 -Nagano,84.03 -Nagano,84.47 -Nagano,84.52 -Shiga,84.93 -Nagano,84.99 -Shiga,85.02 -Nagano,85.28 -Nagano,85.35 -Shiga,85.71 -Shiga,85.67 -Nagano,85.16 -Nagano,85.28 -Nagano
max_2,83.73 -Kumamoto,83.84 -Shiga,83.98 -Shiga,84.24 -Kumamoto,84.36 -Kumamoto,84.63 -Shiga,84.89 -Nagano,84.96 -Shiga,85.15 -Shiga,85.34 -Nagano,85.54 -Kyoto,85.6 -Shiga,85.09 -Shiga,85.19 -Shiga
max_3,83.68 -Fukui,83.74 -Fukui,83.8 -Fukui,84.2 -Shiga,84.35 -Nagano,84.53 -Kumamoto,84.74 -Kyoto,84.89 -Kyoto,85.08 -Kumamoto,85.22 -Nara,85.51 -Nagano,85.39 -Kyoto,84.68 -Okayama,85.09 -Nara
Japan,– 82.95 –,– 82.71 –,– 83.21 –,– 83.43 –,– 83.68 –,– 83.92 –,– 84.1 –,– 84.22 –,– 84.32 –,– 84.46 –,– 84.71 –,– 84.58 –,– 84.1 –,– 84.14 –
Okinawa,– 83.24 –,– 83.0 –,– 83.6 –,– 83.65 –,– 83.56 –,– 83.95 –,– 83.97 –,– 84.06 –,– 84.32 –,– 84.26 –,– 84.52 –,– 83.99 –,– 83.26 –,– 83.39 –
min_3,82.17 -Iwate,80.53 -Fukushima,82.59 -Iwate,82.62 -Akita,82.83 -Akita,83.18 -Iwate,83.29 -Iwate,83.35 -Iwate,83.47 -Fukushima,83.58 -Fukushima,83.96 -Fukushima,83.96 -Gumma,83.23 -Fukushima,83.23 -Akita
min_2,82.11 -Akita,75.17 -Iwate,82.32 -Tochigi,82.57 -Tokushima,82.82 -Tochigi,83.09 -Fukushima,83.11 -Akita,83.23 -Akita,83.41 -Iwate,83.5 -Iwate,83.93 -Ibaraki,83.59 -Fukushima,83.18 -Iwate,83.19 -Iwate
min,81.33 -Aomori,74.51 -Miyagi,81.5 -Aomori,81.91 -Aomori,82.05 -Aomori,82.41 -Aomori,82.65 -Aomori,82.74 -Aomori,82.81 -Aomori,82.6 -Aomori,83.09 -Aomori,82.96 -Aomori,82.69 -Aomori,82.22 -Aomori


In [13]:
# for male
mal.min_and_max_values(df_m, row_center=['Japan', 'Okinawa'], nmb=3, max_lng=9)

Number of records: 48


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
max,80.82 -Nagano,80.86 -Nagano,81.18 -Shiga,81.37 -Nagano,81.53 -Shiga,81.85 -Nagano,81.97 -Nagano,82.11 -Nagano,82.44 -Nagano,82.55 -Nagano,82.88 -Shiga,82.94 -Nagano,82.4 -Nagano,82.58 -Nagano
max_2,80.51 -Shiga,80.66 -Shiga,80.97 -Nagano,81.0 -Shiga,81.31 -Nagano,81.73 -Shiga,81.93 -Shiga,81.92 -Shiga,82.33 -Shiga,82.54 -Shiga,82.54 -Nagano,82.75 -Shiga,82.18 -Shiga,82.38 -Nara
max_3,80.4 -Fukui,80.54 -Nara,80.6 -Fukui,80.92 -Kumamoto,81.02 -Kyoto,81.44 -Nara,81.71 -Nara,81.86 -Kyoto,82.03 -Kyoto,82.26 -Nara,82.45 -Nara,82.45 -Nara,81.74 -Nara,82.31 -Shiga
Japan,– 79.51 –,– 79.41 –,– 79.91 –,– 80.17 –,– 80.46 –,– 80.72 –,– 80.94 –,– 81.06 –,– 81.22 –,– 81.37 –,– 81.58 –,– 81.49 –,– 81.06 –,– 81.1 –
Okinawa,– 79.34 –,– 79.04 –,– 79.75 –,– 79.97 –,– 79.92 –,– 80.35 –,– 80.3 –,– 80.35 –,– 81.01 –,– 80.66 –,– 80.92 –,– 80.5 –,– 79.91 –,– 79.94 –
min_3,78.45 -Iwate,77.29 -Fukushima,79.01 -Akita,79.0 -Tokushima,79.62 -Iwate,79.74 -Iwate,79.99 -Wakayama,79.85 -Iwate,80.02 -Akita,80.4 -Fukushima,80.87 -Fukushima,80.57 -Fukushima,80.01 -Iwate,79.92 -Akita
min_2,78.18 -Akita,72.3 -Iwate,78.89 -Iwate,78.84 -Akita,79.22 -Akita,79.57 -Akita,79.7 -Akita,79.71 -Akita,79.97 -Iwate,80.14 -Iwate,80.46 -Akita,80.5 -Okinawa,79.91 -Okinawa,79.68 -Iwate
min,77.21 -Aomori,72.22 -Miyagi,77.77 -Aomori,78.16 -Aomori,78.12 -Aomori,78.8 -Aomori,78.99 -Aomori,78.99 -Aomori,79.2 -Aomori,78.94 -Aomori,79.43 -Aomori,79.42 -Aomori,79.32 -Aomori,78.78 -Aomori


In [14]:
# for female
mal.min_and_max_values(df_f, row_center=['Japan', 'Okinawa'], nmb=3, max_lng=9)

Number of records: 48


,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
max,87.12 -Nagano,87.06 -Shimane,87.38 -Okinawa,87.45 -Nagano,87.58 -Shimane,87.91 -Nagano,87.91 -Shiga,87.93 -Okayama,88.14 -Shimane,88.21 -Shimane,88.68 -Tottori,88.36 -Shiga,87.97 -Shiga,88.14 -Shimane
max_2,86.99 -Okinawa,87.04 -Fukui,87.06 -Shimane,87.26 -Shiga,87.5 -Fukui,87.59 -Shimane,87.8 -Okayama,87.9 -Shiga,88.12 -Fukui,88.12 -Okayama,88.53 -Okayama,88.33 -Shimane,87.96 -Shimane,88.06 -Shiga
max_3,86.98 -Shimane,86.93 -Okayama,86.99 -Niigata,87.25 -Okinawa,87.48 -Okayama,87.56 -Okayama,87.69 -Nagano,87.84 -Okinawa,88.04 -Tottori,88.03 -Shiga,88.51 -Kumamoto,88.31 -Nagano,87.91 -Nagano,87.96 -Nagano
Japan,– 86.24 –,– 85.86 –,– 86.36 –,– 86.56 –,– 86.77 –,– 86.97 –,– 87.12 –,– 87.26 –,– 87.31 –,– 87.44 –,– 87.73 –,– 87.6 –,– 87.11 –,– 87.16 –
Okinawa,– 86.99 –,– 86.9 –,– 87.38 –,– 87.25 –,– 87.18 –,– 87.53 –,– 87.67 –,– 87.84 –,– 87.63 –,– 87.92 –,– 88.17 –,– 87.6 –,– 86.78 –,– 87.07 –
min_3,85.62 -Wakayama,83.75 -Fukushima,85.85 -Ibaraki,85.92 -Wakayama,86.19 -Akita,86.33 -Ibaraki,86.37 -Tochigi,86.53 -Fukushima,86.61 -Fukushima,86.71 -Ibaraki,87.07 -Ibaraki,86.91 -Hokkaido,86.31 -Iwate,86.38 -Tochigi
min_2,85.57 -Tochigi,77.98 -Iwate,85.62 -Tochigi,85.85 -Tochigi,85.95 -Tochigi,86.26 -Fukushima,86.26 -Akita,86.36 -Tochigi,86.56 -Tochigi,86.6 -Tochigi,87.05 -Fukushima,86.63 -Fukushima,86.29 -Fukushima,86.22 -Fukushima
min,85.22 -Aomori,76.73 -Miyagi,84.99 -Aomori,85.44 -Aomori,85.77 -Aomori,85.74 -Aomori,86.04 -Aomori,86.26 -Aomori,86.21 -Aomori,86.09 -Aomori,86.58 -Aomori,86.33 -Aomori,85.89 -Aomori,85.55 -Aomori


<br />
<br />

In [16]:
# ls_1 = sorted(df_t.drop(index='Japan').index.to_list())
# print(len(ls_1))
# ls_1[:5]

In [17]:
# ls_2 = ["Aichi", "Akita", "Aomori", "Chiba", "Ehime", "Fukui", "Fukuoka", "Fukushima", "Gifu", "Gunma", "Hiroshima",
#         "Hokkaido", "Hyogo", "Ibaraki", "Ishikawa", "Iwate", "Kagawa", "Kagoshima", "Kanagawa", "Kochi", "Kumamoto",
#         "Kyoto", "Mie", "Miyagi", "Miyazaki", "Nagano", "Nagasaki", "Nara", "Niigata", "Oita", "Okayama", "Okinawa",
#         "Osaka", "Saga", "Saitama", "Shiga", "Shimane", "Shizuoka", "Tochigi", "Tokushima", "Tokyo", "Tottori",
#         "Toyama", "Wakayama", "Yamagata", "Yamaguchi", "Yamanashi"]
# print(len(ls_2))

In [18]:
# mal.compare_lists(ls_1, ls_2)

<br />
<br />

In [20]:
# renaming of provinces according to requirement of the online-service 'MapChart'
df_t.rename(index={
    'Gumma' : 'Gunma'
}, inplace=True)

df_m.rename(index={
    'Gumma' : 'Gunma'
}, inplace=True)

df_f.rename(index={
    'Gumma' : 'Gunma'
}, inplace=True)

<br />
<br />

<h4>Overall statistics</h4>

In [22]:
dd_legend_main = {
    '85.50–85.74' : '007800',
    '85.25–85.49' : '008800',
    '85.00–85.24' : '009800',
    '84.75–84.99' : '00a700',
    '84.50–84.74' : '00b800',
    '84.25–84.49' : '00cb00',
    '84.00–84.24' : '00e000',
    '83.75–83.99' : '00f000',
    '83.50–83.74' : '00ff00',
    '83.25–83.49' : '88ff00',
    '83.00–83.24' : 'b8ff00',
    '82.75–82.99' : 'deff00',
    '82.50–82.74' : 'ffff00',
    '82.25–82.49' : 'ffef00',
    '82.00–82.24' : 'ffe000',
    '81.75–81.99' : 'ffce00',
    '81.50–81.74' : 'ffbc00',
    '81.25–81.49' : 'ffac00',
    '81.00–81.24' : 'ff9c00',
    '80.75–80.99' : 'ff8700',
    '80.50–80.74' : 'ff7400',
    '80.25–80.49' : 'ff5200',
    '80.00–80.24' : 'ff0000',
    '79.75–79.99' : 'e10000',
	'79.50–79.74' : 'c70000',
	'79.25–79.49' : 'af0000',
	'79.00–79.24' : '9e0000',
	'78.75–78.99' : '8b0000',
	'78.50–78.74' : '790000',
	'78.25–78.49' : '670000',
	'78.00–78.24' : '580000'
}

def fn_create_legend_code(dd_legend):
    for k, v in dd_legend.items():
        print(f"{{{{Legend|#{v}|{k}}}}}")  

if CREATE_LEGEND_CODE:
    fn_create_legend_code(dd_legend_main)

{{Legend|#007800|85.50–85.74}}
{{Legend|#008800|85.25–85.49}}
{{Legend|#009800|85.00–85.24}}
{{Legend|#00a700|84.75–84.99}}
{{Legend|#00b800|84.50–84.74}}
{{Legend|#00cb00|84.25–84.49}}
{{Legend|#00e000|84.00–84.24}}
{{Legend|#00f000|83.75–83.99}}
{{Legend|#00ff00|83.50–83.74}}
{{Legend|#88ff00|83.25–83.49}}
{{Legend|#b8ff00|83.00–83.24}}
{{Legend|#deff00|82.75–82.99}}
{{Legend|#ffff00|82.50–82.74}}
{{Legend|#ffef00|82.25–82.49}}
{{Legend|#ffe000|82.00–82.24}}
{{Legend|#ffce00|81.75–81.99}}
{{Legend|#ffbc00|81.50–81.74}}
{{Legend|#ffac00|81.25–81.49}}
{{Legend|#ff9c00|81.00–81.24}}
{{Legend|#ff8700|80.75–80.99}}
{{Legend|#ff7400|80.50–80.74}}
{{Legend|#ff5200|80.25–80.49}}
{{Legend|#ff0000|80.00–80.24}}
{{Legend|#e10000|79.75–79.99}}
{{Legend|#c70000|79.50–79.74}}
{{Legend|#af0000|79.25–79.49}}
{{Legend|#9e0000|79.00–79.24}}
{{Legend|#8b0000|78.75–78.99}}
{{Legend|#790000|78.50–78.74}}
{{Legend|#670000|78.25–78.49}}
{{Legend|#580000|78.00–78.24}}


In [23]:
def filter_df(df, selected_year=SELECTED_YEAR):
    filtered_df = df.loc[:, [selected_year]]   \
                    .sort_values(by=selected_year, ascending=False) \
                    .dropna()

    filtered_df['group_label'] = filtered_df[selected_year].map(lambda x: f"{x * 8 // 2 / 4:.2f}–{x * 8 // 2 / 4 + 0.24:.2f}")
    # filtered_df['group_label'] = filtered_df[selected_year].map(lambda x: f"{x * 10 // 2 / 5:.1f}–{x * 10 // 2 / 5 + 0.19:.2f}")
    
    filtered_df['group_label'] = filtered_df['group_label']
    
    min_value = filtered_df[selected_year].min()
    max_value = filtered_df[selected_year].max()
    
    print(f"           ——— {selected_year} ———")
    print(f"Range: {min_value:.2f} – {max_value:.2f}   " +
          f"({filtered_df[selected_year].idxmin()} – {filtered_df[selected_year].idxmax()})")
    print(f"Number of groups: {filtered_df['group_label'].nunique()}")
    print(f"Number of values: {len(filtered_df)}")

    return filtered_df

df_sel_year = filter_df(df_t, SELECTED_YEAR)
df_sel_year

           ——— 2020 ———
Range: 83.09 – 85.71   (Aomori – Shiga)
Number of groups: 9
Number of values: 48


,2020,group_label
Shiga,85.71,85.50–85.74
Kyoto,85.54,85.50–85.74
Nagano,85.51,85.50–85.74
Nara,85.45,85.25–85.49
Okayama,85.39,85.25–85.49
Kumamoto,85.37,85.25–85.49
Hiroshima,85.37,85.25–85.49
Tottori,85.31,85.25–85.49
Ishikawa,85.25,85.25–85.49
Toyama,85.22,85.00–85.24


In [24]:
def extract_indexes(subdf, dd_legend):
    group_label = subdf['group_label'].iloc[0]
    countries = subdf.index.to_list()
    color = (dd_legend[group_label])
    
    ls_grouping.append(CountryGroup(group_label=group_label, countries=countries, color=color))

    return pd.Series([color, countries], index=['color', 'regions'])


ls_grouping = []
df_grouped = df_sel_year.groupby(['group_label']).apply(extract_indexes, dd_legend = dd_legend_main).loc[::-1]

df_grouped

,color,regions
group_label,,
85.50–85.74,007800,"[Shiga, Kyoto, Nagano]"
85.25–85.49,008800,"[Nara, Okayama, Kumamoto, Hiroshima, Tottori, ..."
85.00–85.24,009800,"[Toyama, Oita, Kanagawa, Gifu, Hyogo, Yamanashi]"
84.75–84.99,00a700,"[Shimane, Tokyo, Fukui, Fukuoka, Saga, Miyagi]"
84.50–84.74,00b800,"[Niigata, Japan, Aichi, Mie, Yamaguchi, Kochi,..."
84.25–84.49,00cb00,"[Wakayama, Yamagata, Tokushima, Miyazaki, Sait..."
84.00–84.24,00e000,"[Osaka, Tochigi, Iwate, Akita]"
83.75–83.99,00f000,"[Fukushima, Ibaraki]"
83.00–83.24,b8ff00,[Aomori]


In [25]:
def create_map_code(ls_grouping, title='', legend_color='#000'):  # (ls_grouping, title='', legend_color='#000'):
    false, true = False, True

    jo = {
        "groups": { },
        "title": title,
        "hidden": [],
        "background": "#ffffff",
        "borders": "#000",
        "legendFont": "Century Gothic",
        "legendFontColor": "#000",
        "legendBorderColor": "#00000000",
        "legendBgColor": "#00000000",
        "legendWidth": 150,
        "legendBoxShape": "square",
        "areBordersShown": true,
        "defaultColor": "#d1dbdd",
        "labelsColor": "#6a0707",
        "labelsFont": "Arial",
        "strokeWidth": "medium",
        "areLabelsShown": true,
        "uncoloredScriptColor": "#ffff33",
        "zoomLevel": "1.00",
        "zoomX": "0.00",
        "zoomY": "0.00",
        "v6": true,
        "page": "japan",
        "legendPosition": "custom",
        "legendX": 84,
        "legendY": 36,
        "legendSize": "medium",
        "legendTranslateX": "0.00",
        "legendStatus": "show",
        "scalingPatterns": true,
        "legendRowsSameColor": true,
        "legendColumnCount": 1
    }

    for group_label, color, regions in ls_grouping[::-1]:
        jo["groups"][f"#{color}"] = {"label": group_label.replace('.00', '.0').replace('.25', '¼').replace('.50', '.5').replace('.75', '¾').replace('–', ' - '),
                                     "paths": [region.replace(' ', '_') for region in regions]}       

    return jo


jo = create_map_code(ls_grouping, title=f'{SELECTED_YEAR}', legend_color='#333333')

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{SELECTED_YEAR}.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

<br />
<br />

<h4>Statistics for males</h4>

In [27]:
dd_legend_male = {
    '83.00–83.24' : '020f19',
	'82.75–82.99' : '042136',
	'82.50–82.74' : '062f4c',
	'82.25–82.49' : '073a5e',
	'82.00–82.24' : '08436e',
	'81.75–81.99' : '0a4e7f',
	'81.50–81.74' : '0b578d',
	'81.25–81.49' : '0c609c',
	'81.00–81.24' : '0d68a9',
	'80.75–80.99' : '0e71b7',
	'80.50–80.74' : '107ac6',
	'80.25–80.49' : '1182d4',
	'80.00–80.24' : '128be1',
	'79.75–79.99' : '1c95ec',
	'79.50–79.74' : '2e9eee',
	'79.25–79.49' : '40a6ef',
	'79.00–79.24' : '51aef0',
	'78.75–78.99' : '61b5f2',
	'78.50–78.74' : '6ebcf3',
	'78.25–78.49' : '80c4f4',
	'78.00–78.24' : '92ccf6',
	'77.75–77.99' : 'a2d4f7',
	'77.50–77.74' : 'b3dbf8',
	'77.25–77.49' : 'c2e3fa',
	'77.00–77.24' : 'd4ebfb',
	'76.75–76.99' : 'e4f2fc',
	'76.50–76.74' : 'edf7fd',
	'76.25–76.49' : 'f4fafe',
	'76.00–76.24' : 'f8fbfe'
}

if CREATE_LEGEND_CODE:
    fn_create_legend_code(dd_legend_male)

{{Legend|#020f19|83.00–83.24}}
{{Legend|#042136|82.75–82.99}}
{{Legend|#062f4c|82.50–82.74}}
{{Legend|#073a5e|82.25–82.49}}
{{Legend|#08436e|82.00–82.24}}
{{Legend|#0a4e7f|81.75–81.99}}
{{Legend|#0b578d|81.50–81.74}}
{{Legend|#0c609c|81.25–81.49}}
{{Legend|#0d68a9|81.00–81.24}}
{{Legend|#0e71b7|80.75–80.99}}
{{Legend|#107ac6|80.50–80.74}}
{{Legend|#1182d4|80.25–80.49}}
{{Legend|#128be1|80.00–80.24}}
{{Legend|#1c95ec|79.75–79.99}}
{{Legend|#2e9eee|79.50–79.74}}
{{Legend|#40a6ef|79.25–79.49}}
{{Legend|#51aef0|79.00–79.24}}
{{Legend|#61b5f2|78.75–78.99}}
{{Legend|#6ebcf3|78.50–78.74}}
{{Legend|#80c4f4|78.25–78.49}}
{{Legend|#92ccf6|78.00–78.24}}
{{Legend|#a2d4f7|77.75–77.99}}
{{Legend|#b3dbf8|77.50–77.74}}
{{Legend|#c2e3fa|77.25–77.49}}
{{Legend|#d4ebfb|77.00–77.24}}
{{Legend|#e4f2fc|76.75–76.99}}
{{Legend|#edf7fd|76.50–76.74}}
{{Legend|#f4fafe|76.25–76.49}}
{{Legend|#f8fbfe|76.00–76.24}}


In [28]:
df_sel_year = filter_df(df_m, SELECTED_YEAR)
df_sel_year

           ——— 2020 ———
Range: 79.43 – 82.88   (Aomori – Shiga)
Number of groups: 11
Number of values: 48


,2020,group_label
Shiga,82.88,82.75–82.99
Nagano,82.54,82.50–82.74
Nara,82.45,82.25–82.49
Kyoto,82.34,82.25–82.49
Gifu,82.24,82.00–82.24
Hiroshima,82.17,82.00–82.24
Toyama,82.08,82.00–82.24
Okayama,82.08,82.00–82.24
Kanagawa,82.06,82.00–82.24
Ishikawa,81.97,81.75–81.99


In [29]:
ls_grouping = []
df_grouped = df_sel_year.groupby(['group_label']).apply(extract_indexes, dd_legend = dd_legend_male).loc[::-1]

In [30]:
jo = create_map_code(ls_grouping, title=f'{SELECTED_YEAR}: ♂', legend_color='#333333')  # legend_color='#0000bf'

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{SELECTED_YEAR}, male.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

<br />
<br />

<h4>Statistics for females</h4>

In [32]:
dd_legend_female = {
    '89.00–89.24' : '1e0204',
    '88.75–88.99' : '41050a',
    '88.50–88.74' : '60070f',
    '88.25–88.49' : '840a14',
    '88.00–88.24' : 'a30d19',
    '87.75–87.99' : 'bd0f25',
    '87.50–87.74' : 'd81132',
    '87.25–87.49' : 'ec183b',
    '87.00–87.24' : 'ef3b59',
    '86.75–86.99' : 'f1536e',
    '86.50–86.74' : 'f2657d',
    '86.25–86.49' : 'f3768b',
    '86.00–86.24' : 'f58698',
    '85.75–85.99' : 'f696a6',
    '85.50–85.74' : 'f7a5b2',
    '85.25–85.49' : 'f8b3be',
    '85.00–85.24' : 'f9bfc6',
    '84.75–84.99' : 'facdd3',
    '84.50–84.74' : 'fbd8db',
    '84.25–84.49' : 'fce2e4',
    '84.00–84.24' : 'fdebed',
    '83.75–83.99' : 'fef3f4',
    '83.50–83.74' : 'fef8f9',
    '83.25–83.49' : 'fefafa',
}

if CREATE_LEGEND_CODE:
    fn_create_legend_code(dd_legend_female)

{{Legend|#1e0204|89.00–89.24}}
{{Legend|#41050a|88.75–88.99}}
{{Legend|#60070f|88.50–88.74}}
{{Legend|#840a14|88.25–88.49}}
{{Legend|#a30d19|88.00–88.24}}
{{Legend|#bd0f25|87.75–87.99}}
{{Legend|#d81132|87.50–87.74}}
{{Legend|#ec183b|87.25–87.49}}
{{Legend|#ef3b59|87.00–87.24}}
{{Legend|#f1536e|86.75–86.99}}
{{Legend|#f2657d|86.50–86.74}}
{{Legend|#f3768b|86.25–86.49}}
{{Legend|#f58698|86.00–86.24}}
{{Legend|#f696a6|85.75–85.99}}
{{Legend|#f7a5b2|85.50–85.74}}
{{Legend|#f8b3be|85.25–85.49}}
{{Legend|#f9bfc6|85.00–85.24}}
{{Legend|#facdd3|84.75–84.99}}
{{Legend|#fbd8db|84.50–84.74}}
{{Legend|#fce2e4|84.25–84.49}}
{{Legend|#fdebed|84.00–84.24}}
{{Legend|#fef3f4|83.75–83.99}}
{{Legend|#fef8f9|83.50–83.74}}
{{Legend|#fefafa|83.25–83.49}}


In [33]:
df_sel_year = filter_df(df_f, SELECTED_YEAR)
df_sel_year

           ——— 2020 ———
Range: 86.58 – 88.68   (Aomori – Tottori)
Number of groups: 8
Number of values: 48


,2020,group_label
Tottori,88.68,88.50–88.74
Okayama,88.53,88.50–88.74
Kumamoto,88.51,88.50–88.74
Kyoto,88.49,88.25–88.49
Shiga,88.39,88.25–88.49
Nagano,88.39,88.25–88.49
Hiroshima,88.39,88.25–88.49
Oita,88.37,88.25–88.49
Ishikawa,88.34,88.25–88.49
Yamanashi,88.24,88.00–88.24


In [34]:
ls_grouping = []
df_grouped = df_sel_year.groupby(['group_label']).apply(extract_indexes, dd_legend = dd_legend_female).loc[::-1]

In [35]:
jo = create_map_code(ls_grouping, title=f'{SELECTED_YEAR}: ♀', legend_color='#333333')  # legend_color='#c20000'

pretty_jo = json.dumps(jo, indent=2, ensure_ascii=False)

with open(f"output/map_JSON -{SELECTED_YEAR}, female.txt", 'w', encoding="utf-8") as fh:
    fh.write(pretty_jo)

In [ ]:
assert False

<br />
<br />

exploration:

In [ ]:
print(round(85.71 - 82.22, 2))
print(int(round(85.75 - 82.00, 2) * 4))
print(int(round(85.8 - 82.2, 2) * 5))

In [ ]:
print(round(82.88 - 78.78, 2))
print(int(round(83.00 - 78.75, 2) * 4))
print(int(round(83.0 - 78.6, 2) * 5))

In [ ]:
print(round(88.68 - 85.55, 2))
print(int(round(88.75 - 85.50, 2) * 4))
print(int(round(88.8 - 85.4, 2) * 5))

<br />
<br />

creation of technical dictionaries:

In [ ]:
ls = [
    '007800',
    '008800',
    '009800',
    '00a700',
    '00b800',
    '00cb00',
    '00e000',
    '00f000',
    '00ff00',
    '88ff00',
    'b8ff00',
    'deff00',
    'ffff00',
    'ffef00',
    'ffe000',
    'ffce00',
    'ffbc00',
    'ffac00',
    'ff9c00',
    'ff8700',
    'ff7400',
    'ff5200',
    'ff0000',
    'e10000',
    'c70000',
    'af0000',
    '9e0000',
    '8b0000',
    '790000',
    '670000',
    '580000'
]

def print_legend(ls, age):
    for col in ls:
        print(f"    '{age :.2f}–{age+0.24 :.2f}' : '{col}',")
        age -= 0.25

print_legend(ls, age=87.00)

In [ ]:
ls = [
    '020f19',
    '042136',
    '062f4c',
    '073a5e',
    '08436e',
    '0a4e7f',
    '0b578d',
    '0c609c',
    '0d68a9',
    '0e71b7',
    '107ac6',
    '1182d4',
    '128be1',
    '1c95ec',
    '2e9eee',
    '40a6ef',
    '51aef0',
    '61b5f2',
    '6ebcf3',
    '80c4f4',
    '92ccf6',
    'a2d4f7',
    'b3dbf8',
    'c2e3fa',
    'd4ebfb',
    'e4f2fc',
    'edf7fd',
    'f4fafe',
    'f8fbfe'
]

print_legend(ls, age=84.00)

In [ ]:
# v1
ls = [
    '1e0204',
    '3f050a',
    '5a070e',
    '700911',
    '860a15',
    '9c0c18',
    'ad0e1b',
    'bd0f25',
    'cd1030',
    'df1234',
    'ec2042',
    'ef3b59',
    'f04f6a',
    'f15d76',
    'f26a80',
    'f3768b',
    'f48295',
    'f58fa0',
    'f69aaa',
    'f7a4b2',
    'f8acb9',
    'f9b5c0',
    'f9bec5',
    'fac7cd',
    'fbcfd3',
    'fbd7da',
    'fcdfe2',
    'fde7e9',
    'fdeeef',
    'fef4f4',
    'fef8f9'
]

print_legend(ls, age=90.00)  # age=90.00

In [ ]:
# v2
ls = [
    '1e0204',
    '41050a',
    '60070f',
    '840a14',
    'a30d19',
    'bd0f25',
    'd81132',
    'ec183b',
    'ef3b59',
    'f1536e',
    'f2657d',
    'f3768b',
    'f58698',
    'f696a6',
    'f7a5b2',
    'f8b3be',
    'f9bfc6',
    'facdd3',
    'fbd8db',
    'fce2e4',
    'fdebed',
    'fef3f4',
    'fef8f9',
    'fefafa'
]

print_legend(ls, age=89.00)